# Plot rental prices to map of Zurich

In [9]:
# Libraries
import os
import folium
import openpyxl
import platform
import pandas as pd
import geopandas as gpd
import pprint

# Docker only (setting the path using the environmental variable PROJ_LIB)
#if platform.system() == 'Linux':
#    import os
#    import conda
#    conda_file_dir = conda.__file__
#    conda_dir = conda_file_dir.split('lib')[0]
#    proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
#    os.environ["PROJ_LIB"] = proj_lib
#else:
#    pass

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Polygonmap als .json-File
polys = gpd.read_file("../data/stzh.adm_statistische_quartiere_map.json")

# Structure and type
print("nrows, ncols", polys.shape)
print("-------------------------------------------------------")
print("Type:", type(polys))

# Object 'polys' is a GeoDataFrame
polys.head()

nrows, ncols (34, 7)
-------------------------------------------------------
Type: <class 'geopandas.geodataframe.GeoDataFrame'>


,objectid,objid,qnr,qname,knr,kname,geometry
0,1,34,73,Hirslanden,7,Kreis 7,"POLYGON ((8.55670 47.36407, 8.55670 47.36406, ..."
1,2,33,83,Weinegg,8,Kreis 8,"POLYGON ((8.55671 47.36405, 8.55670 47.36405, ..."
2,3,32,82,Mühlebach,8,Kreis 8,"POLYGON ((8.55423 47.36456, 8.55422 47.36458, ..."
3,4,31,81,Seefeld,8,Kreis 8,"POLYGON ((8.54795 47.36500, 8.54781 47.36501, ..."
4,7,16,41,Werd,4,Kreis 4,"POLYGON ((8.53301 47.37394, 8.53310 47.37405, ..."


In [11]:
rentalPrices = pd.read_csv('../data/rentalprices_temp.csv', sep=';')
## only for 3 bedroom flats
rooms = 3
rentalPrices = rentalPrices[rentalPrices.Zimmer == rooms]
rentalPrices.head()

,Unnamed: 0,Raumeinheit,GliederungSort,Gliederung,Zimmer,GemeinnuetzigSort,GemeinnuetzigLang,EinheitSort,EinheitLang,mean,Grundgesamtheit,Sample1,Sample2
376,1722,Quartiere,11,Rathaus,3,0,Alle Wohnungen,1,Wohnung,2391.0,490,169,40
377,1723,Quartiere,12,Hochschulen,3,0,Alle Wohnungen,1,Wohnung,2612.0,50,18,22
378,1724,Quartiere,13,Lindenhof,3,0,Alle Wohnungen,1,Wohnung,2974.0,181,54,31
379,1725,Quartiere,14,City,3,0,Alle Wohnungen,1,Wohnung,2410.0,115,31,18
380,1726,Quartiere,21,Wollishofen,3,0,Alle Wohnungen,1,Wohnung,1864.0,3345,1016,77


In [12]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
cp = folium.Choropleth(
    geo_data=polys,
    data=rentalPrices,
    columns=["Gliederung", "mean"],
    key_on='feature.properties.qname',
    name='polys',
    fill_color='OrRd',
).add_to(m)

# creating a state indexed version of the dataframe so we can lookup values
rental_prices_indexed = rentalPrices.set_index('Gliederung')


# looping thru the geojson object and adding a new property(unemployment)
# and assigning a value from our dataframe
for q in cp.geojson.data['features']:
    q['properties']['price'] = rental_prices_indexed.loc[q['properties']['qname'], 'mean']
  
# and finally adding a tooltip/hover to the choropleth's geojson
folium.GeoJsonTooltip(fields=['qname', 'price'], aliases=['Quartier', 'Durchschnittlicher Mietpreis ('+str(rooms)+' Zimmer) in CHF']).add_to(cp.geojson)
  
folium.LayerControl().add_to(m)
m